In [30]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import pandas as pd
import yaml
import os

with open("config.yml", 'r') as ymlfile:
    cfg = yaml.load(ymlfile)

#import data
bulkdir = "Data/Slideseq/" + cfg['default']['slideseqfolder'] + "/results/Bulk"
X = pd.read_csv(os.path.join(bulkdir, "X_bead.csv"))
b = pd.read_csv(os.path.join(bulkdir, "b_bead.csv"))
cell_type_names = X.columns[1:]
X = X.iloc[:,1:].to_numpy()
b = b.iloc[:,1].to_numpy()

/Users/dcable/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  


In [21]:
sum(b)

35

In [31]:
#define the model
class Model(object):
    def __init__(self):
        self.N = len(cell_type_names)
        self.w = tf.Variable([1/self.N for i in range(self.N)])
        self.min_pred = 1e-7

    def __call__(self, x):
        return tf.math.maximum(self.min_pred,tf.tensordot(x[0:(self.N)],tf.abs(self.w),1))
    
    def predict_batch(self, x):
        return tf.math.maximum(self.min_pred,tf.linalg.matvec(x[:,0:(self.N)],tf.abs(self.w))) 

def loss(predicted_y, target_y):
    loss = (1 - target_y)*predicted_y - target_y*tf.math.log(1-tf.math.exp(-predicted_y))
    return tf.math.reduce_sum(loss)

model = Model()

In [58]:
#create the loss function
X_var = tf.compat.v1.placeholder("float32") 
Y_var = tf.compat.v1.placeholder("float32") 
my_loss = loss(model(X_var),Y_var)
batch_loss = loss(model.predict_batch(X_var),Y_var)
#optimizer = tf.train.GradientDescentOptimizer(3e-4).minimize(my_loss)
optimizer = tf.compat.v1.train.AdagradOptimizer(3e-2).minimize(my_loss)
#optimizer = tf.compat.v1.train.AdadeltaOptimizer(3e-1).minimize(my_loss)
init = tf.global_variables_initializer() 


In [60]:
#train the model
training_epochs = 200
with tf.Session() as sess: 
     
    # Initializing the Variables 
    sess.run(init) 
      
    # Iterating through all the epochs 
    for epoch in range(training_epochs): 
          
        # Feeding each data point into the optimizer using Feed Dictionary 
        for (_x, _y) in zip(X, b): 
            sess.run(optimizer, feed_dict = {X_var : _x, Y_var : _y}) 
          
        # Displaying the result after every 5 epochs 
        if (epoch + 1) % 5 == 0: 
            # Calculating the cost a every epoch 
            c = sess.run(batch_loss, feed_dict = {X_var : X, Y_var : b}) 
            print("Epoch", (epoch + 1), ": cost =", c, "w =", sess.run(model.w)) 
      
    # Storing necessary values to be used outside the Session 
    training_cost = sess.run(batch_loss, feed_dict = {X_var : X, Y_var : b}) 
    weight = sess.run(model.w) 


Epoch 5 : cost = 525.239 w = [-0.05318838  0.10888277  0.2797118  -0.004394   -0.00542696  0.07393
 -0.01160634 -0.0031391  -0.07668236  0.00144637  0.01667845 -0.00414193
 -0.00034618 -0.00622844  0.03855028 -0.01911726  0.00184282]
Epoch 10 : cost = 523.05145 w = [-5.6703825e-02  1.0844120e-01  3.2707417e-01 -3.1542999e-03
  4.4646128e-03  5.3742506e-02 -9.0992423e-03  2.9841887e-03
 -7.3247336e-02  9.7102829e-04  1.8798696e-02 -3.0801934e-03
  2.3349328e-04 -4.4091186e-03  1.9970205e-02 -1.9396313e-02
 -1.2460656e-03]
Epoch 15 : cost = 521.98346 w = [-5.83356433e-02  1.05713055e-01  3.55560809e-01 -2.57580634e-03
  3.79768061e-03  3.94120142e-02 -7.64299463e-03  2.61579826e-03
 -7.13761002e-02 -7.77379784e-04  2.26838328e-02 -2.57998612e-03
 -1.82658012e-04 -3.65558756e-03  9.45497025e-03 -1.97235011e-02
 -9.89268883e-04]
Epoch 20 : cost = 521.353 w = [-5.9211567e-02  1.0228334e-01  3.7538418e-01  2.2187924e-03
  3.3691507e-03  2.8351743e-02  6.6603902e-03  2.3574766e-03
 -6.9758520

Epoch 145 : cost = 519.3939 w = [-6.1512191e-02  4.6096712e-02  4.7685811e-01  7.1904744e-04
 -2.4155343e-03  9.5035491e-04 -2.4676842e-03 -1.0114972e-03
 -4.4609372e-02  2.3497113e-04  3.9286952e-02 -8.9228433e-04
 -5.8585967e-05 -1.1156496e-02 -1.8886010e-03 -2.0869115e-02
 -3.0835668e-04]
Epoch 150 : cost = 519.38245 w = [-6.1533902e-02  4.5199227e-02  4.7793958e-01 -7.0836616e-04
 -2.4184485e-03 -9.3534752e-04  2.4253395e-03 -1.0005187e-03
 -4.4331677e-02  2.3074629e-04  3.9449666e-02  8.7810255e-04
 -5.7609835e-05 -1.1228492e-02  1.8583782e-03 -2.0871380e-02
  3.0205882e-04]
Epoch 155 : cost = 519.37146 w = [-6.1553732e-02  4.4347584e-02  4.7896460e-01  6.9775275e-04
 -2.4208189e-03  9.2067529e-04  2.3874107e-03 -9.8998612e-04
 -4.4072025e-02  2.2690895e-04  3.9603017e-02  8.6416770e-04
  5.6613400e-05 -1.1295705e-02 -1.8280809e-03 -2.0873383e-02
 -2.9827363e-04]
Epoch 160 : cost = 519.36163 w = [-6.1571818e-02  4.3538287e-02  4.7993243e-01  6.8599964e-04
 -2.4219707e-03  9.064398

In [24]:
with tf.Session() as sess: 
    true_w = [0 for i in range(model.N)]
    true_w[11] =1
    print(true_w)
    model.w = tf.constant(true_w, dtype=tf.float32)
    true_cost = loss(model.predict_batch(tf.constant(X,dtype=tf.float32)),tf.constant(b,dtype=tf.float32)).eval()
    print("Truth: cost =",true_cost,"true_w =",true_w)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]
Truth: cost = 186.79779 true_w = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]


In [21]:
weight_df = pd.DataFrame(abs(weight),index=cell_type_names,columns=["Weight"])
weight_df.to_csv(os.path.join(bulkdir, "weights.csv"))


In [47]:
sum(abs(weight))

0.7110364104009932